# Readme 

# Fonctions utiles 

In [1]:
import os
import json

from nltk import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
import math

In [2]:
# Fonction pour nettoyer le texte des caractères spéciaux
def clean_text(text, characters):
    c_text = text
    for char in characters:
        c_text = c_text.replace(char, " ")
    c_text = c_text.lower()
    return(c_text)

In [3]:
# Calcul de document frequency
def df(data):
    df = {}
    for desc in data.values():
        for token in set(word_tokenize(desc)):
            stem = stemmer.stem(token)
            if stem in df:
                df[stem] += 1
            else:
                df[stem] = 1
    return df

In [4]:
# Récupération des mots stemmés
def stem_words(data):
    stem_dict = {}
    for k, desc in data.items():
        tokens = word_tokenize(desc)
        for t in tokens:
            token_stem = stemmer.stem(t)
            if token_stem in stem_dict:
                stem_dict[token_stem].append(t)
            else:
                stem_dict[token_stem] = [t]

    for k, v in stem_dict.items():
        stem_dict[k] = list(set(stem_dict[k]))
    
    return stem_dict

In [5]:
# Tri des mots par leur valeur de tf-idf.
def sorted_tfidf(tf_idf, stem_dict, nb_mots=15):
    sorted_tfidf = {}
    for clus, dico in tf_idf.items():
        list_temp = [(j, i) for i, j in dico.items()]  # on met la valeur du tfidf du mot en 1er attribut
        list_temp = sorted(list_temp, reverse=True)  # on ordonne la liste suivant les valeurs de tfidf décroissante
        tfidf_clus = [(j,i) for i, j in list_temp]
        sorted_clus = []
        for (token, j) in tfidf_clus:
            token = stem_dict[token]
            sorted_clus.append([token, j])
        sorted_tfidf[clus] = sorted_clus[0:nb_mots]
        
    final_tfidf = {}
    for k, v in sorted_tfidf.items():
        final_tfidf[k] = []
        for item in v:
            final_tfidf[k].append(item[0][0])
    return final_tfidf

In [6]:
# tf-idf for all texts
def tf_idf(data, nb_mots):
    nb_texts = len(data.keys())
    term_freq = df(data)

    tf_idf ={}
    for k, desc in data.items():
        tf_idf_clus = {}
        tokens = word_tokenize(desc)
        tokens = [stemmer.stem(t) for t in tokens]
        frequencies = FreqDist(tokens)

        for token, freq in frequencies.items():
            tf_idf_clus[token] = freq * math.log( nb_texts / (1 + term_freq.get(token, 0)) )

        tf_idf[k] = tf_idf_clus

    stem_dict = stem_words(data)
    return sorted_tfidf(tf_idf,stem_dict,nb_mots)

# Main

### Préparation des données

In [18]:
# Préparation des chemins utiles et chargement des données

# Prend le chemin du working directory
dir_path = os.getcwd()

# Ensemble des textes
data_dir_path = os.path.join(dir_path, 'data.json')
with open(data_dir_path, 'r') as f:
    data = json.load(f)

# Créer si besoin un dossier output et prépare le chemin pour 'tf_idf.json'
output_dir_path = os.path.join(dir_path, 'output.json')

### Options

In [8]:
# Stemmer
stemmer = SnowballStemmer('french')

# Choix du nombre de mots pour le texte
nb_mots = 15

# Charactères à retirer
special_chars = [",", "\n",".", "'", ";", "\\n",'"',")","(","»","«"]

### Nettoyage des textes

In [9]:
# Nettoyage de la ponctuation dans les définitions des métiers
clean_data = {}
for k, desc in data.items():
    desc_str = str(desc)
    desc_c = clean_text(desc_str, special_chars)
    clean_data[k]=desc_c
data = clean_data

### Processing

In [10]:
# TF-IDF et sélection des meilleurs résultats
tf_idf = tf_idf(data, nb_mots)

### Affichages des résultats

In [12]:
# Afficher les résultats de tous les texts
#print(tf_idf)

In [13]:
# Afficher les résultats d'un text en particulier
key = "A1101"
print(tf_idf[key])

['forestière', 'agricoles', 'engins', 'coopérative', 'r']


In [19]:
# Copie du dictionnaire au format JSON
with open(output_dir_path, 'w') as f:
    json.dump(tf_idf, f, indent=4)